# SuperAI Season 4 - Level 2 Hackathon - Forest Type Double Classifier

## Explore Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('./datasets/train.csv' , index_col='id')
test_df = pd.read_csv('./datasets/test.csv' , index_col='id')

## Create NON-DEF (MDF / DDF) / DEF 

In [3]:
DEF_df = train_df[train_df['nforest_type'] == 'DEF']
NON_DEF_df = train_df[train_df['nforest_type'] != 'DEF']

In [4]:
NON_DEF_df.to_csv('./datasets/MDF_DDF.csv')

In [5]:
NON_DEF_df['nforest_type'] = 'NON_DEF'
DEF_df['nforest_type'] = 'DEF'

C:\Users\teehe\AppData\Local\Temp\ipykernel_22388\3381345259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NON_DEF_df['nforest_type'] = 'NON_DEF'
C:\Users\teehe\AppData\Local\Temp\ipykernel_22388\3381345259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DEF_df['nforest_type'] = 'DEF'


In [6]:
NON_DEF_df

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9,nforest_type
id,,,,,,,,,,,,,
2002,293,1927,1038,278,475,453,987,1773,2184,1900,2343,3039,NON_DEF
3212,197,1598,697,201,347,228,682,1982,2449,2254,2685,2690,NON_DEF
13312,929,1975,1031,982,1020,856,1220,2051,2421,2392,2671,2683,NON_DEF
17020,132,1560,689,189,408,175,609,2117,2907,3024,3005,2955,NON_DEF
5967,241,1944,1131,362,538,487,918,1549,1844,1702,2077,2043,NON_DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9185,374,1940,1054,382,565,498,977,1678,1929,2109,2291,2100,NON_DEF
13977,1983,3602,2720,1622,1782,1766,2314,3488,3900,3924,4097,6053,NON_DEF
755,940,2007,1148,975,1080,968,1252,1780,1983,1942,2247,2170,NON_DEF


In [7]:
DEF_df

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9,nforest_type
id,,,,,,,,,,,,,
14932,701,1776,895,689,905,684,1187,2504,2836,2839,3172,3103,DEF
14572,156,1121,547,264,472,362,699,1673,1967,2182,2260,2277,DEF
7504,536,1591,777,686,760,624,1049,2152,2600,2665,2964,2777,DEF
10489,1551,1519,718,1730,1764,1514,1719,2267,2554,2598,2743,2743,DEF
7995,313,1696,864,366,568,411,849,1965,2387,2439,2827,2759,DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3762,182,1336,574,188,296,145,571,2000,2473,2243,2884,2844,DEF
7804,1,1753,820,106,431,301,840,2091,2472,2762,2914,2440,DEF
5566,281,1837,835,342,718,285,975,2801,3556,3637,3801,3148,DEF


In [8]:
DEF_NONDEF_df = pd.concat([NON_DEF_df , DEF_df])
DEF_NONDEF_df.sort_index().to_csv('./datasets/DEF_NONDEF.csv')

## DEF_NONDEF Classifier

In [9]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np

In [10]:
DEF_NONDEF_df = pd.read_csv('./datasets/DEF_NONDEF.csv' , index_col='id')
test_df = pd.read_csv('./datasets/test_cleaned.csv', index_col = 'id')

In [11]:
def add_features_for_DEF_NONDEF(row) :

    row['NDVI'] = (row['b8'] - row['b4']) / (row['b8'] + row['b4'])
    row['EVI'] = 2.5 * ((row['b8'] - row['b4']) / (row['b8'] + 6 * row['b4'] - 7.5 * row['b2'] + 1.01))
    row['NDWI '] = (row['b3'] - row['b8']) / (row['b3'] + row['b8'])
    row['SAVI '] = (row['b8'] - row['b4']) * (1 + 0.5) / (row['b8'] + row['b4'] + 0.5)
    row['MSAVI'] = (2 * row['b8'] + 1 - ( (2 * row['b8'] + 1) ** 2 - 8 * (row['b8'] - row['b4'])) ** (1 / 2)) / 2
    row['GNDVI '] = (row['b8'] - row['b3']) / (row['b8'] + row['b3'])
    row['RENDVI '] = (row['b8'] - row['b5']) / (row['b8'] + row['b5'])
    row['NDMI '] = (row['b8'] - row['b11']) / (row['b8'] + row['b11'])
    row['GRVI'] = (row['b3'] - row['b4']) / (row['b3'] + row['b4'])
    row['TVI'] = ( (row['b8'] - row['b4']) / (row['b8'] + row['b4'] + 0.5) ) ** (1 / 2)
    row['MCARI'] = ((row['b5'] - row['b4']) - 0.2 * (row['b5'] - row['b3'])) / (row['b5'] / row['b4'])
    row['BSI'] =  ((row['b11'] + row['b4']) - (row['b8'] + row['b2'])) / ((row['b11'] + row['b4']) + (row['b8'] + row['b2']))
    row['NBR'] = (row['b8'] - row['b12']) / (row['b8'] + row['b12'])
    row['MSI'] = row['b11'] / row['b8']

    return row


In [12]:
DEF_NONDEF_df = DEF_NONDEF_df.apply(add_features_for_DEF_NONDEF , axis = 1)
test_df = test_df.apply(add_features_for_DEF_NONDEF , axis = 1)

In [13]:
DEF_NONDEF_df['nforest_type'].value_counts()

nforest_type
NON_DEF    10468
DEF         2585
Name: count, dtype: int64

In [14]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 42 , sampling_strategy= 'all')

DEF_NONDEF_df , label_df  = smote.fit_resample(DEF_NONDEF_df.drop(columns=['nforest_type']) , DEF_NONDEF_df['nforest_type'])
DEF_NONDEF_df = DEF_NONDEF_df.join(label_df)

c:\anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py:110: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py", line 199, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\anaconda\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\anaconda\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^

In [15]:
DEF_NONDEF_df['nforest_type'].value_counts()

nforest_type
NON_DEF    10468
DEF        10468
Name: count, dtype: int64

In [16]:
label = 'nforest_type'
predictor = TabularPredictor(label = label).fit(DEF_NONDEF_df)

No path specified. Models will be saved in: "AutogluonModels\ag-20240608_063616"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240608_063616"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:      

[1000]	valid_set's binary_error: 0.0864374
[2000]	valid_set's binary_error: 0.0687679
[3000]	valid_set's binary_error: 0.0592168
[4000]	valid_set's binary_error: 0.0573066
[5000]	valid_set's binary_error: 0.056829
[6000]	valid_set's binary_error: 0.0563515


	0.9465	 = Validation score   (accuracy)
	10.7s	 = Training   runtime
	0.3s	 = Validation runtime
Fitting model: LightGBM ...
Available Memory: 507 MB
Estimated GBM model size: 0 MB
	0.8214	 = Validation score   (accuracy)
	0.47s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9207	 = Validation score   (accuracy)
	3.93s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9226	 = Validation score   (accuracy)
	6.06s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ...
	0.9451	 = Validation score   (accuracy)
	62.1s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9222	 = Validation score   (accuracy)
	1.17s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9217	 = Validation score   (accuracy)
	1.14s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9136	 = Validation s

[1000]	valid_set's binary_error: 0.0582617


	0.9446	 = Validation score   (accuracy)
	8.29s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.308, 'KNeighborsDist': 0.154, 'RandomForestGini': 0.154, 'CatBoost': 0.154, 'LightGBMXT': 0.077, 'RandomForestEntr': 0.077, 'ExtraTreesGini': 0.077}
	0.9599	 = Validation score   (accuracy)
	0.15s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 277.12s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20240608_063616")


In [17]:
predictor.refit_full()

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: KNeighborsUnif_FULL ...
	0.03s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: KNeighborsDist_FULL ...
	0.03s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMXT_FULL ...
	7.37s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_FULL ...
	0.38s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestGini_FULL ...
	3.51s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestEntr_FULL ...
	4.89s	 = Training   runtime
Fitting 1 L1 models ...
Fitting mode

{'KNeighborsUnif': 'KNeighborsUnif_FULL',
 'KNeighborsDist': 'KNeighborsDist_FULL',
 'LightGBMXT': 'LightGBMXT_FULL',
 'LightGBM': 'LightGBM_FULL',
 'RandomForestGini': 'RandomForestGini_FULL',
 'RandomForestEntr': 'RandomForestEntr_FULL',
 'CatBoost': 'CatBoost_FULL',
 'ExtraTreesGini': 'ExtraTreesGini_FULL',
 'ExtraTreesEntr': 'ExtraTreesEntr_FULL',
 'NeuralNetFastAI': 'NeuralNetFastAI_FULL',
 'XGBoost': 'XGBoost_FULL',
 'NeuralNetTorch': 'NeuralNetTorch_FULL',
 'LightGBMLarge': 'LightGBMLarge_FULL',
 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}

In [18]:
predictor.fit_pseudolabel(test_df)

Given test_data for pseudo labeling did not contain labels. AutoGluon will assign pseudo labels to data and use it for extra training data...
Beginning iteration 1 of pseudolabeling out of max 3
Pseudolabeling algorithm confidently assigned pseudolabels to 450 rows of data on iteration 1. Adding to train data
Fitting 13 L1 models ...
Fitting model: KNeighborsUnif_PSEUDO_1 ...
	0.8997	 = Validation score   (accuracy)
	0.04s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: KNeighborsDist_PSEUDO_1 ...
	0.9093	 = Validation score   (accuracy)
	0.04s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: LightGBMXT_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.0840497
[2000]	valid_set's binary_error: 0.0644699
[3000]	valid_set's binary_error: 0.0596944
[4000]	valid_set's binary_error: 0.056829
[5000]	valid_set's binary_error: 0.056829


	0.9446	 = Validation score   (accuracy)
	10.71s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: LightGBM_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.0625597
[2000]	valid_set's binary_error: 0.056829
[3000]	valid_set's binary_error: 0.0558739
[4000]	valid_set's binary_error: 0.0477555
[5000]	valid_set's binary_error: 0.0496657


	0.9527	 = Validation score   (accuracy)
	9.08s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: RandomForestGini_PSEUDO_1 ...
	0.9212	 = Validation score   (accuracy)
	3.04s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestEntr_PSEUDO_1 ...
	0.9222	 = Validation score   (accuracy)
	4.06s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost_PSEUDO_1 ...
	0.9422	 = Validation score   (accuracy)
	68.63s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini_PSEUDO_1 ...
	0.9183	 = Validation score   (accuracy)
	1.59s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr_PSEUDO_1 ...
	0.9226	 = Validation score   (accuracy)
	1.63s	 = Training   runtime
	0.27s	 = Validation runtime
Fitting model: NeuralNetFastAI_PSEUDO_1 ...
	0.9198	 = Validation score   (accuracy)
	40.39s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: XGBoost_PSEUDO_1 ...
	0.9422	 = Va

[1000]	valid_set's binary_error: 0.0587393
[2000]	valid_set's binary_error: 0.0534862


	0.9475	 = Validation score   (accuracy)
	17.72s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: WeightedEnsemble_L2_PSEUDO_1 ...
	Ensemble Weights: {'LightGBM_PSEUDO_1': 0.5, 'NeuralNetTorch_PSEUDO_1': 0.5}
	0.9585	 = Validation score   (accuracy)
	0.13s	 = Training   runtime
	0.0s	 = Validation runtime
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20240608_063616")
Pseudolabeling algorithm changed validation score from: 0.9598853868194842, to: 0.9598853868194842 using evaluation metric: accuracy


In [19]:
prediction = predictor.predict(test_df)

In [20]:
feature_importances = predictor.feature_importance(DEF_NONDEF_df)
feature_importances

Computing feature importance via permutation shuffling for 26 features using 5000 rows with 5 shuffle sets...
	202.3s	= Expected runtime (40.46s per shuffle set)
	71.58s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
b11,0.25124,0.004172,9.123557e-09,5,0.259831,0.242649
b8_a,0.16972,0.002938,1.077205e-08,5,0.175769,0.163671
b9,0.11052,0.002941,6.015070e-08,5,0.116576,0.104464
b6,0.09864,0.005149,8.874563e-07,5,0.109241,0.088039
b12,0.08704,0.005642,2.106181e-06,5,0.098656,0.075424
b5,0.08236,0.001571,1.587799e-08,5,0.085595,0.079125
b7,0.07664,0.004607,1.559867e-06,5,0.086127,0.067153
b1,0.06904,0.003618,9.014391e-07,5,0.076489,0.061591
b8,0.06040,0.005002,5.593030e-06,5,0.070699,0.050101
GRVI,0.05792,0.002625,5.050901e-07,5,0.063325,0.052515


In [21]:
feature_importances.to_csv('./features/DEF_NONDEF_features_importance.csv')

In [22]:
prediction.value_counts()

nforest_type
NON_DEF    3012
DEF         988
Name: count, dtype: int64

In [23]:
prediction = prediction.sort_index()

In [24]:
prediction

id
1        NON_DEF
2        NON_DEF
5        NON_DEF
7        NON_DEF
12           DEF
          ...   
17039    NON_DEF
17042    NON_DEF
17043    NON_DEF
17047    NON_DEF
17052    NON_DEF
Name: nforest_type, Length: 4000, dtype: object

In [25]:
prediction_DEF_ONLY = prediction[prediction == 'DEF']
prediction_DEF_ONLY

id
12       DEF
19       DEF
29       DEF
50       DEF
51       DEF
        ... 
17013    DEF
17024    DEF
17026    DEF
17032    DEF
17034    DEF
Name: nforest_type, Length: 988, dtype: object

In [26]:
test_df = test_df.sort_index()

## MDF_DDF Classifier

In [27]:
test_df = pd.read_csv('./datasets/test_cleaned.csv', index_col='id')

In [28]:
test_df = test_df[prediction == 'NON_DEF']

C:\Users\teehe\AppData\Local\Temp\ipykernel_22388\254591184.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = test_df[prediction == 'NON_DEF']


In [29]:
MDF_DDF_df = pd.read_csv('./datasets/MDF_DDF.csv' , index_col='id')

In [30]:
def add_features_for_MDF_DDF(row) :

    row['NDVI'] = (row['b8'] - row['b4']) / (row['b8'] + row['b4'])
    row['EVI'] = 2.5 * ((row['b8'] - row['b4']) / (row['b8'] + 6 * row['b4'] - 7.5 * row['b2'] + 1.01))
    row['NDWI '] = (row['b3'] - row['b8']) / (row['b3'] + row['b8'])
    row['SAVI '] = (row['b8'] - row['b4']) * (1 + 0.5) / (row['b8'] + row['b4'] + 0.5)
    row['MSAVI'] = (2 * row['b8'] + 1 - ( (2 * row['b8'] + 1) ** 2 - 8 * (row['b8'] - row['b4'])) ** (1 / 2)) / 2
    row['GNDVI '] = (row['b8'] - row['b3']) / (row['b8'] + row['b3'])
    row['RENDVI '] = (row['b8'] - row['b5']) / (row['b8'] + row['b5'])
    row['NDMI '] = (row['b8'] - row['b11']) / (row['b8'] + row['b11'])
    row['GRVI'] = (row['b3'] - row['b4']) / (row['b3'] + row['b4'])
    row['TVI'] = ( (row['b8'] - row['b4']) / (row['b8'] + row['b4'] + 0.5) ) ** (1 / 2)
    row['MCARI'] = ((row['b5'] - row['b4']) - 0.2 * (row['b5'] - row['b3'])) / (row['b5'] / row['b4'])
    row['BSI'] =  ((row['b11'] + row['b4']) - (row['b8'] + row['b2'])) / ((row['b11'] + row['b4']) + (row['b8'] + row['b2']))
    row['NBR'] = (row['b8'] - row['b12']) / (row['b8'] + row['b12'])
    row['MSI'] = row['b11'] / row['b8']

    return row

In [31]:
MDF_DDF_df = MDF_DDF_df.apply(add_features_for_MDF_DDF , axis = 1)
test_df = test_df.apply(add_features_for_MDF_DDF , axis = 1)

In [32]:
MDF_DDF_df['nforest_type'].value_counts()

nforest_type
MDF    5865
DDF    4603
Name: count, dtype: int64

In [33]:
from imblearn.over_sampling import SMOTE 

smote = SMOTE(random_state = 42 , sampling_strategy= 'all')

MDF_DDF_df , label_df  = smote.fit_resample(MDF_DDF_df.drop(columns=['nforest_type']) , MDF_DDF_df['nforest_type'])
MDF_DDF_df = MDF_DDF_df.join(label_df)

In [34]:
MDF_DDF_df['nforest_type'].value_counts()

nforest_type
MDF    5865
DDF    5865
Name: count, dtype: int64

In [35]:
predictor_for_MDF_DDF = TabularPredictor(label = label).fit(MDF_DDF_df)

No path specified. Models will be saved in: "AutogluonModels\ag-20240608_065123"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240608_065123"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:      

[1000]	valid_set's binary_error: 0.244672


	0.7621	 = Validation score   (accuracy)
	1.86s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.236999


	0.7681	 = Validation score   (accuracy)
	1.41s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7613	 = Validation score   (accuracy)
	1.93s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7579	 = Validation score   (accuracy)
	2.76s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost ...
	0.7442	 = Validation score   (accuracy)
	9.13s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7656	 = Validation score   (accuracy)
	1.07s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7621	 = Validation score   (accuracy)
	1.1s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.7596	 = Validation score   (accuracy)
	12.51s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	0.7613	 = Validation score   (accuracy)
	4.38s	 = Training   runtime
	0.03s	

In [36]:
predictor_for_MDF_DDF.refit_full()

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: KNeighborsUnif_FULL ...
	0.02s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: KNeighborsDist_FULL ...
	0.02s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMXT_FULL ...
	1.47s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_FULL ...
	1.12s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestGini_FULL ...
	2.13s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestEntr_FULL ...
	3.04s	 = Training   runtime
Fitting 1 L1 models ...
Fitting mode

{'KNeighborsUnif': 'KNeighborsUnif_FULL',
 'KNeighborsDist': 'KNeighborsDist_FULL',
 'LightGBMXT': 'LightGBMXT_FULL',
 'LightGBM': 'LightGBM_FULL',
 'RandomForestGini': 'RandomForestGini_FULL',
 'RandomForestEntr': 'RandomForestEntr_FULL',
 'CatBoost': 'CatBoost_FULL',
 'ExtraTreesGini': 'ExtraTreesGini_FULL',
 'ExtraTreesEntr': 'ExtraTreesEntr_FULL',
 'NeuralNetFastAI': 'NeuralNetFastAI_FULL',
 'XGBoost': 'XGBoost_FULL',
 'NeuralNetTorch': 'NeuralNetTorch_FULL',
 'LightGBMLarge': 'LightGBMLarge_FULL',
 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}

In [37]:
predictor_for_MDF_DDF.fit_pseudolabel(test_df)

Given test_data for pseudo labeling did not contain labels. AutoGluon will assign pseudo labels to data and use it for extra training data...
Beginning iteration 1 of pseudolabeling out of max 3
Pseudolabeling algorithm confidently assigned pseudolabels to 98 rows of data on iteration 1. Adding to train data
Fitting 13 L1 models ...
Fitting model: KNeighborsUnif_PSEUDO_1 ...
	0.7272	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: KNeighborsDist_PSEUDO_1 ...
	0.7425	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: LightGBMXT_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.244672


	0.7596	 = Validation score   (accuracy)
	2.09s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.232737


	0.7707	 = Validation score   (accuracy)
	3.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini_PSEUDO_1 ...
	0.7587	 = Validation score   (accuracy)
	1.87s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr_PSEUDO_1 ...
	0.7587	 = Validation score   (accuracy)
	2.59s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost_PSEUDO_1 ...
	0.7604	 = Validation score   (accuracy)
	17.23s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini_PSEUDO_1 ...
	0.7673	 = Validation score   (accuracy)
	1.31s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesEntr_PSEUDO_1 ...
	0.7664	 = Validation score   (accuracy)
	1.15s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: NeuralNetFastAI_PSEUDO_1 ...
	0.7639	 = Validation score   (accuracy)
	12.01s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost_PSEUDO_1 ...
	0.7613	 = Vali

In [38]:
prediction_MDF_DDF = predictor_for_MDF_DDF.predict(add_features_for_MDF_DDF( test_df) )

In [39]:
feature_importances = predictor_for_MDF_DDF.feature_importance(MDF_DDF_df)
feature_importances

Computing feature importance via permutation shuffling for 26 features using 5000 rows with 5 shuffle sets...
	91.18s	= Expected runtime (18.24s per shuffle set)
	32.58s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
b9,0.29700,0.007118,3.955062e-08,5,0.311655,0.282345
b7,0.21980,0.004656,2.415067e-08,5,0.229387,0.210213
b8,0.21920,0.005381,4.355793e-08,5,0.230280,0.208120
b12,0.20924,0.004853,3.468958e-08,5,0.219232,0.199248
b11,0.20688,0.005681,6.815888e-08,5,0.218577,0.195183
b8_a,0.19356,0.007736,3.055580e-07,5,0.209489,0.177631
b5,0.19172,0.001677,7.022599e-10,5,0.195173,0.188267
b6,0.17880,0.007277,3.285856e-07,5,0.193784,0.163816
b1,0.13400,0.004790,1.955282e-07,5,0.143862,0.124138
b3,0.13316,0.000792,1.505168e-10,5,0.134792,0.131528


In [40]:
feature_importances.to_csv('./features/MDF_DDF_features_importance.csv')

In [41]:
prediction_MDF_DDF.value_counts()

nforest_type
DDF    1611
MDF    1401
Name: count, dtype: int64

In [42]:
submission_df = pd.concat([prediction_DEF_ONLY , prediction_MDF_DDF])
submission_df = submission_df.sort_index()
submission_df

id
1        MDF
2        DDF
5        MDF
7        DDF
12       DEF
        ... 
17039    DDF
17042    DDF
17043    DDF
17047    MDF
17052    DDF
Name: nforest_type, Length: 4000, dtype: object

In [43]:
submission_df.value_counts()

nforest_type
DDF    1611
MDF    1401
DEF     988
Name: count, dtype: int64

In [44]:
submission_path = './submissions'
submission_df.to_csv(f'{submission_path}/submission_addfeatures_SMOTE_pseudolabeling_double.csv')